## Read files into a pandas DataFrame

In [57]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/cellphoneaccessory"

In [58]:
import fastai
from fastai.text import *
from fastai import *

In [59]:
ls {DATAPATH}

inter/
reviews_Cell_Phones_and_Accessories_5.json
reviews_Cell_Phones_and_Accessories_5.json.gz


In [60]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(f'{DATAPATH}/reviews_Cell_Phones_and_Accessories_5.json.gz')

In [61]:
df['overall'].value_counts()

5.0    108664
4.0     39993
3.0     21439
1.0     13279
2.0     11064
Name: overall, dtype: int64

In [62]:
df['label']=np.where(df['overall']>3,1,np.where(df['overall']<3,0,-1))

In [63]:
df['label'].value_counts()

 1    148657
 0     24343
-1     21439
Name: label, dtype: int64

In [64]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,label
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014",1
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014",1
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014",1
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013",1
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013",1


In [65]:
df['timestamp']=pd.to_datetime(df['unixReviewTime'],unit='s')

In [66]:
df['timestamp'].describe()

count                  194439
unique                   2830
top       2014-01-02 00:00:00
freq                      450
first     2001-02-22 00:00:00
last      2014-07-23 00:00:00
Name: timestamp, dtype: object

In [67]:
df['timestamp'].dt.year.value_counts().sort_index()

2001        1
2002        1
2003        2
2004       44
2005      130
2006      240
2007      389
2008      677
2009     1307
2010     3431
2011     9591
2012    30342
2013    93239
2014    55045
Name: timestamp, dtype: int64

In [68]:
df['year']=df['timestamp'].dt.year

In [69]:
df['year'].value_counts()

2013    93239
2014    55045
2012    30342
2011     9591
2010     3431
2009     1307
2008      677
2007      389
2006      240
2005      130
2004       44
2003        2
2002        1
2001        1
Name: year, dtype: int64

In [70]:
pd.crosstab(df['overall'],df['year'])

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
overall,,,,,,,,,,,,,,
1.0,0,0,0,4,11,28,24,52,139,264,763,2258,6225,3511
2.0,0,0,0,7,18,32,29,51,95,233,660,1850,5296,2793
3.0,0,0,0,2,15,33,50,73,152,428,1028,3407,10806,5445
4.0,0,0,0,16,39,75,111,197,302,858,2257,6412,19175,10551
5.0,1,1,2,15,47,72,175,304,619,1648,4883,16415,51737,32745


In [71]:
# Take the years after 2014 out as full year is not there

In [72]:
#df=df.loc[df['year']<2014]

In [73]:
#df.shape

In [74]:
# Remove the old data also out

In [75]:
#df=df.loc[df['year']>2011]

In [76]:
#df.shape

In [77]:
#df['label'].head()

In [78]:
df['label'].value_counts()

 1    148657
 0     24343
-1     21439
Name: label, dtype: int64

In [79]:
df=df.loc[df['label']!=-1]

In [80]:
df.shape

(173000, 12)

In [81]:
df['label']=1-df['label']

In [82]:
df['label'].value_counts()

0    148657
1     24343
Name: label, dtype: int64

In [83]:
from sklearn.model_selection import train_test_split

In [82]:
df_train,df_valid=train_test_split(df,train_size=0.5,test_size=0.5,random_state=11,shuffle=True,stratify=df['label'])

In [83]:
df_train['dstype']='train'
df_valid['dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [84]:
df_train.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,label,dstype
154964,A3KWJ8B4CC5ECL,B00BW6GH76,karen,"[0, 0]",I was initially torn between the Neo Hybrid an...,5.0,Slim Armor vs. Neo Hybrid,1377820800,"08 30, 2013",0,train
42712,A1MHQBDNXI9X4U,B004WMDTHS,Lawrence bowman,"[0, 0]",Nice fit like it. Not really strong but it is ...,4.0,I like It,1354233600,"11 30, 2012",0,train
13443,A1NNMN8XNH4264,B002YFDRHW,K. Myers,"[5, 6]",I bought the New Trent IMP880 a year ago (now ...,4.0,"Battery works fine, but iPad / iPhone cable is...",1319587200,"10 26, 2011",0,train
93385,A4KU0XJNBH674,B0088YDF7K,Army O3Guy,"[0, 0]","Great customer service, and great product. The...",5.0,2nd Time is a Charm,1352505600,"11 10, 2012",0,train
16442,A1I7OF3NNJEEXP,B003KWM9BS,Karen Chhem,"[0, 0]",bought this for my galaxy note 2 it works wond...,5.0,love it.,1389139200,"01 8, 2014",0,train


In [85]:
df=pd.concat([df_train,df_valid],axis=0)

In [86]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,label,dstype
154964,A3KWJ8B4CC5ECL,B00BW6GH76,karen,"[0, 0]",I was initially torn between the Neo Hybrid an...,5.0,Slim Armor vs. Neo Hybrid,1377820800,"08 30, 2013",0,train
42712,A1MHQBDNXI9X4U,B004WMDTHS,Lawrence bowman,"[0, 0]",Nice fit like it. Not really strong but it is ...,4.0,I like It,1354233600,"11 30, 2012",0,train
13443,A1NNMN8XNH4264,B002YFDRHW,K. Myers,"[5, 6]",I bought the New Trent IMP880 a year ago (now ...,4.0,"Battery works fine, but iPad / iPhone cable is...",1319587200,"10 26, 2011",0,train
93385,A4KU0XJNBH674,B0088YDF7K,Army O3Guy,"[0, 0]","Great customer service, and great product. The...",5.0,2nd Time is a Charm,1352505600,"11 10, 2012",0,train
16442,A1I7OF3NNJEEXP,B003KWM9BS,Karen Chhem,"[0, 0]",bought this for my galaxy note 2 it works wond...,5.0,love it.,1389139200,"01 8, 2014",0,train


In [87]:
df.rename(mapper={'reviewText':'text'},axis=1,inplace=True)

In [88]:
df=df[['text','label','dstype']]

In [89]:
df.head()

,text,label,dstype
154964,I was initially torn between the Neo Hybrid an...,0,train
42712,Nice fit like it. Not really strong but it is ...,0,train
13443,I bought the New Trent IMP880 a year ago (now ...,0,train
93385,"Great customer service, and great product. The...",0,train
16442,bought this for my galaxy note 2 it works wond...,0,train


## Cross-Validation

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
df_train,df_valid=train_test_split(df,train_size=0.9,test_size=0.1,random_state=11,shuffle=True)

In [92]:
df.shape,df_train.shape,df_valid.shape

((173000, 3), (155700, 3), (17300, 3))

In [93]:
df.head()

,text,label,dstype
154964,I was initially torn between the Neo Hybrid an...,0,train
42712,Nice fit like it. Not really strong but it is ...,0,train
13443,I bought the New Trent IMP880 a year ago (now ...,0,train
93385,"Great customer service, and great product. The...",0,train
16442,bought this for my galaxy note 2 it works wond...,0,train


In [94]:
df_train_file=df_train.loc[:,['label','text']]
df_valid_file=df_valid.loc[:,['label','text']]

In [95]:
df_train_file.to_csv(f'{DATAPATH}/inter/df_train.csv',index=False)
df_valid_file.to_csv(f'{DATAPATH}/inter/df_valid.csv',index=False)



## Tokenize and Numericalize

In [96]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df, n_lbls=1):
    labels = df.iloc[:, range(n_lbls)].values.astype(np.int64)

    texts = f'\n{BOS} {FLD} 1 ' + df.iloc[:,n_lbls].astype(str)
    for i in range(n_lbls + 1, len(df.columns)):
        texts += f' {FLD} {i - n_lbls} ' + df[i].astype(str)
    texts = texts.apply(fixup).values.astype(str)

    tokenizer = Tokenizer(n_cpus=10)
    tokop=tokenizer.process_all(texts)
    return tokop, list(labels)


def get_all(df, n_lbls):
    tok, labels = [], []
    #import pdb
    #pdb.set_trace()
    for i, txt in enumerate(df):
        tok_, labels_ = get_texts(txt, n_lbls)
        tok += tok_
        labels += labels_
    return tok, labels

In [97]:
chunksize=24000
chunk_train=pd.read_csv(f'{DATAPATH}/inter/df_train.csv',chunksize=chunksize)
chunk_valid=pd.read_csv(f'{DATAPATH}/inter/df_valid.csv',chunksize=chunksize)



In [98]:
train_tokens, train_labels = get_all(chunk_train, 1)
valid_tokens, valid_labels = get_all(chunk_valid, 1)


In [99]:
df_train['words']=train_tokens
df_valid['words']=valid_tokens


/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [100]:
freq = Counter(p for o in train_tokens for p in o)
freq.most_common(25)

[('xxmaj', 967025),
 ('.', 801214),
 ('the', 783007),
 ('i', 470653),
 (',', 469233),
 ('it', 434791),
 ('and', 399359),
 ('a', 366226),
 ('to', 361382),
 ('is', 261888),
 ('this', 217437),
 ('xxup', 216354),
 ('of', 189536),
 ('for', 189372),
 ('my', 187154),
 ('1', 165098),
 (' \n ', 155700),
 ('xxbos', 155700),
 ('xxfld', 155700),
 ('that', 155363),
 ('in', 146501),
 ('on', 145128),
 ('phone', 144257),
 ('with', 141468),
 ('you', 130233)]

In [101]:
max_vocab = 60000
min_freq = 2

itos = [o for o, c in freq.most_common(max_vocab) if c > min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [102]:
stoi = collections.defaultdict(lambda: 0, { v: k for k, v in enumerate(itos) })
len(itos)

30238

In [103]:
trn_lm = np.array([ [stoi[o] for o in p] for p in train_tokens ])
val_lm = np.array([ [stoi[o] for o in p] for p in valid_tokens ])


In [104]:
df_train['tokens']=trn_lm
df_valid['tokens']=val_lm

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [105]:
pickle.dump([df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm],open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','wb'))

In [88]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))